# Experiments名を指定して、学習ジョブを発行

Python 3(PyTorch 1.10 Python 3.8 CPU Optimized)で動作確認

# MNIST Training using PyTorch

参考：wandb

https://github.com/wandb/examples/blob/master/examples/pytorch/pytorch-mnist-sagemaker/pytorch_mnist.ipynb

SageMaker sample

https://github.com/aws/amazon-sagemaker-examples/blob/main/hyperparameter_tuning/pytorch_mnist/hpo_pytorch_mnist.ipynb

In [ ]:
!pip install ipywidgets widgetsnbextension # studioの場合は必要。要カーネル再起動

* torch / torchvision : 1.11.0 / 0.5.0 -> NG
* torch / torchvision : 1.10.0 / 0.5.0 -> NG
* torch / torchvision : 1.5.0 / 0.5.0 -> OK
* torch / torchvision : 1.7.0 / 0.5.0 -> OK
* torch / torchvision : 1.9.0 / 0.10.0 -> NG
* torch / torchvision : 1.10.2 / 0.11.3 -> NG
* torch / torchvision : 1.9.1 / 0.10.1 -> NG
* torch / torchvision : 1.8.0 / 0.9.0 -> OK

In [ ]:
!pip install -U torch==1.8.0 # 1.4.0で動作確認済み # 1.11.0 NG

In [ ]:
!pip install torchvision==0.9.0 # 0.5.0で動作確認済み

In [ ]:
!pip freeze | grep torch

カーネル再起動してください。

In [ ]:
!pip install wandb -qqq

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)

---

## Background

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using PyTorch.

MNISTは、手書きの数字を分類するために広く使われているデータセットである。このデータセットは、手書き数字の28x28ピクセルのグレースケール画像70,000枚から構成される。このデータセットは60,000枚の学習画像と10,000枚のテスト画像に分割される。10個のクラスがあります（10桁の数字にそれぞれ1つずつ）。このチュートリアルでは、PyTorchを使ってSageMaker上でMNISTモデルを学習・テストする方法を紹介します。


For more information about the PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.

---

## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by creating a SageMaker session and specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).


In [ ]:
#!pip install -U sagemaker==2.87.0

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-mnist-wandb-studio'

role = sagemaker.get_execution_role()

In [ ]:
sagemaker.__version__

## Data
### Getting the data



In [ ]:
from torchvision.datasets import MNIST
from torchvision import transforms

MNIST.mirrors = ["https://sagemaker-sample-files.s3.amazonaws.com/datasets/image/MNIST/"]

MNIST(
    'data',
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )
)

### Uploading the data to S3
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.


In [ ]:
inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

In [ ]:
inputs

## Train
### Training script
The `mnist.py` script provides all the code we need for training and hosting a SageMaker model (`model_fn` function to load a model).
The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to.
  These artifacts are uploaded to S3 for model hosting.
* `SM_NUM_GPUS`: The number of gpus available in the current container.
* `SM_CURRENT_HOST`: The name of the current container on the container network.
* `SM_HOSTS`: JSON encoded list containing all the hosts .

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's `fit()` method, the following will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

Because the SageMaker imports the training script, you should put your training code in a main guard (``if __name__=='__main__':``) if you are using the same script to host your model as we do in this example, so that SageMaker does not inadvertently run your training code at the wrong point in execution.

For example, the script run by this notebook:

In [ ]:
!pygmentize src/mnist.py

### Run training in SageMaker

The `PyTorch` class allows us to run our training function as a training job on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters. In this case we are going to run our training job on 2 ```ml.c4.xlarge``` instances. But this example can be ran on one or multiple, cpu or gpu instances ([full list of available instances](https://aws.amazon.com/sagemaker/pricing/instance-types/)). The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the `mnist.py` script above.


In [ ]:
import wandb
wandb.login()
settings = wandb.setup().settings
current_api_key = wandb.wandb_lib.apikey.api_key(settings=settings)

In [ ]:
from sagemaker.pytorch import PyTorch
'''
estimator = PyTorch(entry_point='mnist.py',
                    source_dir="src",
                    role=role,
                    py_version='py3',
                    framework_version='1.8.0',
                    instance_count=1,
                    instance_type='ml.c5.2xlarge',
                    hyperparameters={
                        'epochs': 1,
                        'backend': 'gloo'
                    },
                   environment={"WANDB_API_KEY": current_api_key})
'''

学習ジョブのエスティメーターを定義


In [ ]:
estimator = PyTorch(entry_point='mnist.py',
                    source_dir="src",
                    role=role,
                    py_version='py3',
                    framework_version='1.8.0',
                    instance_count=1,
                    instance_type='ml.c5.2xlarge',
                    hyperparameters={
                        'epochs': 1,
                        'backend': 'gloo'
                    },
                    metric_definitions=[
                        {'Name': 'training/loss', 'Regex': 'Loss: (.*?)#015'},
                        #{'Name': 'Validation Loss', 'Regex': 'val_loss: (.*?);'},
                        {'Name': 'test/loss', 'Regex': 'Average loss: (.*?),'},
                    ],
                    environment={"WANDB_API_KEY": current_api_key})

After we've constructed our `PyTorch` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.


この設定はStepFunctionsでパイプラインを作るとき、MLOpsエンジニア側の作業となります。

In [ ]:
from smexperiments.experiment import Experiment
import boto3
sm = boto3.Session().client('sagemaker')
from smexperiments.tracker import Tracker
import time
from smexperiments.trial import Trial

# experimentの作成
experiment = Experiment.create(experiment_name="mlops-experiment11", 
                               description="Sample Experiments for MLOps.", 
                               sagemaker_boto_client=sm)

# trackerの作成
with Tracker.create(display_name=f"tracker-{int(time.time())}", sagemaker_boto_client=sm) as tracker:
    tracker.log_input(name="input-dataset-dir", media_type="s3/uri", value='s3://mlops/input/')
    tracker.log_input(name="output-dataset-dir", media_type="s3/uri", value='s3://mlops/output/')

# trialの作成
trial_name = f"training-job-{int(time.time())}"
experiments_trial = Trial.create(
    trial_name=trial_name,
    experiment_name=experiment.experiment_name,
    sagemaker_boto_client=sm,
)

# trial_componentの付与
#sample_trial_component = tracker.trial_component
#experiments_trial.add_trial_component(sample_trial_component)

In [ ]:
# trial_componentの付与
estimator_trial_component = tracker.trial_component
experiments_trial.add_trial_component(sample_trial_component)

In [ ]:
estimator.fit({'training': inputs},
    # trialの情報を指定
    experiment_config={
        "ExperimentName": experiment.experiment_name,
        "TrialName": experiments_trial.trial_name,
        "TrialComponentDisplayName": estimator_trial_component.display_name,
    },
)

In [ ]:
#estimator.fit({'training': inputs})

## 複数ジョブ発行

In [ ]:
lrs = [0.01, 0.001, 0.05, 0.1, 0.2]
for lr in lrs:
    estimator = PyTorch(entry_point='mnist.py',
                        source_dir="src",
                        role=role,
                        py_version='py3',
                        framework_version='1.8.0',
                        instance_count=1,
                        instance_type='ml.c5.2xlarge',
                        hyperparameters={
                            'epochs': 10,
                            'backend': 'gloo',
                            'lr': lr,
                        },
                       environment={"WANDB_API_KEY": current_api_key})
    estimator.fit({'training': inputs}, wait=False)